# NLP 데이터셋 API와 BERT의 적용

## 데이터셋 API

In [7]:
!pip install datasets

In [2]:
from datasets import list_datasets
datasets_list = list_datasets()
print(len(datasets_list))

print(', '.join(dataset for dataset in datasets_list))

1188
acronym_identification, ade_corpus_v2, adversarial_qa, aeslc, afrikaans_ner_corpus, ag_news, ai2_arc, air_dialogue, ajgt_twitter_ar, allegro_reviews, allocine, alt, amazon_polarity, amazon_reviews_multi, amazon_us_reviews, ambig_qa, amttl, anli, app_reviews, aqua_rat, aquamuse, ar_cov19, ar_res_reviews, ar_sarcasm, arabic_billion_words, arabic_pos_dialect, arabic_speech_corpus, arcd, arsentd_lev, art, arxiv_dataset, ascent_kb, aslg_pc12, asnq, asset, assin, assin2, atomic, autshumato, babi_qa, banking77, bbaw_egyptian, bbc_hindi_nli, bc2gm_corpus, best2009, bianet, bible_para, big_patent, billsum, bing_coronavirus_query_set, biomrc, blended_skill_talk, blimp, blog_authorship_corpus, bn_hate_speech, bookcorpus, bookcorpusopen, boolq, bprec, break_data, brwac, bsd_ja_en, bswac, c3, c4, cail2018, caner, capes, catalonia_independence, cawac, cbt, cc100, cc_news, ccaligned_multilingual, cdsc, cdt, cfq, chr_en, cifar10, cifar100, circa, civil_comments, clickbait_news_bg, climate_fever, 

In [3]:
from datasets import load_dataset
dataset_glue = load_dataset('glue', 'mrpc') # 2개의 문장이 같은 의미인지 여부를 구별하는 분류 태크스
print(dataset_glue) # 데이터 구조 dict

Reusing dataset glue (C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [4]:
dataset_glue['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [14]:
dataset_glue['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

In [35]:
dataset_T=load_dataset('glue', 'mrpc',split='train')
print(dataset_T)

Reusing dataset glue (C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})


In [6]:
# train 최초의 10%, val 최초의 10%, test 최초의 10%로 데이터 구성
dataset_partial=load_dataset('glue', 'mrpc',split='train[:10%]+validation[:10%]+test[:10%]')
print(dataset_partial)

Reusing dataset glue (C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 580
})


In [7]:
dataset_squad = load_dataset('squad') # question과 context를 입력하여 answer을 위한 시작과 끝 index 찾는 분류 태스크
print(dataset_squad)

Reusing dataset squad (C:\Users\user\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [8]:
dataset_squad10 = load_dataset('squad',split='train[:10%]+validation[:10%]')
print(dataset_squad10)

Reusing dataset squad (C:\Users\user\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 9817
})


In [9]:
dataset_squad10[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

## 사전학습 모형
TFBertForSequenceClassification, TFAutoModelForSequenceClassification

In [16]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import numpy as np

In [17]:
model_c = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
tokenizer_c = BertTokenizer.from_pretrained('bert-base-cased')
print(tokenizer_c(dataset_T[0]['sentence1'], dataset_T[0]['sentence2']))

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': [101, 7277, 2180, 5303, 4806, 1117, 1711, 117, 2292, 1119, 1270, 107, 1103, 7737, 107, 117, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 11336, 6732, 3384, 1106, 1140, 1112, 1178, 107, 1103, 7737, 107, 117, 7277, 2180, 5303, 4806, 1117, 1711, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [36]:
 tokenizer_c.decode(tokenizer_c(dataset_T[0]['sentence1'], dataset_T[0]['sentence2'])['input_ids'])

'[CLS] Amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] Referring to him as only " the witness ", Amrozi accused his brother of deliberately distorting his evidence. [SEP]'

In [37]:
def encode(sentences):
    return tokenizer_c(sentences['sentence1'], sentences['sentence2'], truncation=True, padding='max_length')

dataset_T = dataset_T.map(encode, batched=True)

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dbd55a14d3d0907a.arrow


In [40]:
dataset_T[0:1]

{'attention_mask': [[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [41]:
dataset_T = dataset_T.map(lambda sentences: {'labels': sentences['label']}, batched=True)

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-6f5b5fbf4ac87164.arrow


In [44]:
dataset_T[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [32]:
import tensorflow as tf
dataset_T.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
features = {x: dataset_T[x].to_tensor(default_value=0, shape=[None, tokenizer_c.model_max_length]) for x in ['input_ids', 'token_type_ids', 'attention_mask']}
tfdataset = tf.data.Dataset.from_tensor_slices((features, dataset_T["labels"])).batch(32)
next(iter(tfdataset))

({'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[  101,  7277,  2180, ...,     0,     0,     0],
         [  101, 10684,  2599, ...,     0,     0,     0],
         [  101,  1220,  1125, ...,     0,     0,     0],
         ...,
         [  101,  1109,  2026, ...,     0,     0,     0],
         [  101, 22263,  1107, ...,     0,     0,     0],
         [  101,   142,  1813, ...,     0,     0,     0]], dtype=int64)>,
  'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int64)>,
  'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
  

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
model_c.compile(optimizer=opt, loss=loss_fn, metrics=["accuracy"])
model_c.fit(tfdataset, epochs=3)

Epoch 1/3
  2/115 [..............................] - ETA: 5:08:59 - loss: 1.0373 - accuracy: 0.4297

In [20]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
model_auto = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['dropout_113', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dataset.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
features_auto = {x: dataset[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'token_type_ids', 'attention_mask']}
tfdataset_auto = tf.data.Dataset.from_tensor_slices((features_auto, dataset["labels"])).batch(32)
next(iter(tfdataset_auto))

({'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[  101,  7277,  2180, ...,     0,     0,     0],
         [  101, 10684,  2599, ...,     0,     0,     0],
         [  101,  1220,  1125, ...,     0,     0,     0],
         ...,
         [  101,  1109,  2026, ...,     0,     0,     0],
         [  101, 22263,  1107, ...,     0,     0,     0],
         [  101,   142,  1813, ...,     0,     0,     0]], dtype=int64)>,
  'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int64)>,
  'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
  

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
model_auto.compile(optimizer=opt, loss=loss_fn, metrics=["accuracy"])
model_auto.fit(tfdataset, epochs=3)

Epoch 1/3
  1/115 [..............................] - ETA: 3:45:45 - loss: 0.7574 - accuracy: 0.5312

## 개인 소장 local 파일 + TFBertForSequenceClassification

예시

In [ ]:
>>> from datasets import load_dataset
>>> dataset = load_dataset('csv', data_files='my_file.csv')
>>> dataset = load_dataset('csv', data_files=['my_file_1.csv', 'my_file_2.csv', 'my_file_3.csv'])
>>> dataset = load_dataset('csv', data_files={'train': ['my_train_file_1.csv', 'my_train_file_2.csv'],
>>>                                           'test': 'my_test_file.csv'})

In [1]:
#!pip install gdown
#!gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
from datasets import load_dataset
data_imdb = load_dataset('csv', data_files='./imdbs.csv', split='train')

Using custom data configuration default-8c9f44ffec2a67b3
Reusing dataset csv (C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [2]:
data_imdb[0]

{'text': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due 

In [3]:
data_imdb.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [4]:
# train, test 데이터셋 분리
data_imdb = data_imdb.train_test_split(test_size=0.3)

Loading cached split indices for dataset at C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-5b342961a9a313a3.arrow and C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-855da866fb156bb9.arrow


In [5]:
data_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30
    })
})

In [6]:
train_set = data_imdb['train']
test_set = data_imdb['test']
train_set.shape

(70, 2)

In [7]:
train_set[0]

{'text': 'If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!!',
 'label': 1}

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

train_data = train_set.map(encode, batched=True)
test_data=test_set.map(encode,batched=True)

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-9b0e3d6f484c526b.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-554586f673d296ba.arrow


In [10]:
train_data[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [11]:
train_data = train_data.map(lambda examples: {'labels': examples['label']}, batched=True)
test_data = test_data.map(lambda examples: {'labels': examples['label']}, batched=True)

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-a525421f01f71a7e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\csv\default-8c9f44ffec2a67b3\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-b57f5b0894c4a3fb.arrow


In [12]:
train_data[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [13]:
import tensorflow as tf
train_data.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'labels'])
features_train = {x: train_data[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
tfdata_train = tf.data.Dataset.from_tensor_slices((features_train, train_data["labels"])).batch(32)
next(iter(tfdata_train))

({'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[  101,  2065,  2017, ...,     0,     0,     0],
         [  101,  2054,  3047, ...,     0,     0,     0],
         [  101,  1045,  2245, ...,     0,     0,     0],
         ...,
         [  101,  2023,  2001, ...,     0,     0,     0],
         [  101, 28353,  5149, ...,     0,     0,     0],
         [  101,  1045,  2074, ...,  9930,  2005,   102]], dtype=int64)>,
  'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 1, 1, 1]], dtype=int64)>},
 <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 1], dtype=int64)>)

In [14]:
test_data.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'labels'])
features_test = {x: test_data[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
tfdata_test = tf.data.Dataset.from_tensor_slices((features_test, test_data["labels"])).batch(32)

In [15]:
from transformers import TFBertForSequenceClassification
model_tf=TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model_tf.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model_tf.fit(tfdata_train, epochs=3,validation_data=tfdata_test)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
3/3 [==============================] - 379s 123s/step - loss: 6.7235 - accuracy: 0.5865 - val_loss: 5.1416 - val_accuracy: 0.3333
Epoch 2/3
3/3 [==============================] - 407s 117s/step - loss: 6.7191 - accuracy: 0.4356 - val_loss: 5.1416 - val_accuracy: 0.3333
Epoch 3/3
3/3 [==============================] - 423s 120s/step - loss: 6.4966 - accuracy: 0.4356 - val_loss: 7.6539 - val_accuracy: 0.6667


## TFDistilBertModel 이전학습

In [67]:
from transformers import TFDistilBertModel

tfdist_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

input_ids_in = tf.keras.layers.Input(shape=(512,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32') 

# 최종 출력 embedding
embedding_layer = tfdist_model(input_ids_in, attention_mask=input_masks_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(100, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='softmax')(X)

model = tf.keras.Model(inputs=[input_ids_in,input_masks_in], outputs = X)

# 첫 3개의 layer들의 모수를 고정
for layer in model.layers[:3]:
  layer.trainable = False

model.summary()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti ((None, 512, 768),)  66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model_1[0][0

In [62]:
tf_traindata = tf.data.Dataset.from_tensor_slices(((train_data['input_ids'],train_data['attention_mask']), 
                                                   train_data["labels"])).shuffle(50).batch(8,drop_remainder=True)
next(iter(tf_traindata))

((<tf.Tensor: shape=(8, 512), dtype=int32, numpy=
  array([[  101,  2023,  2001, ...,     0,     0,     0],
         [  101,  4459,  9881, ...,     0,     0,     0],
         [  101,  2588, 10523, ...,     0,     0,     0],
         ...,
         [  101,  2004,  1037, ...,     0,     0,     0],
         [  101,  2061, 10047, ..., 11393,  1006,   102],
         [  101,  1045,  2245, ...,     0,     0,     0]])>,
  <tf.Tensor: shape=(8, 512), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 0, 0, 0]])>),
 <tf.Tensor: shape=(8,), dtype=int32, numpy=array([0, 1, 0, 0, 1, 1, 0, 1])>)

In [63]:
tf_testdata = tf.data.Dataset.from_tensor_slices(((test_data['input_ids'],test_data['attention_mask']), 
                                                   test_data["labels"])).shuffle(20).batch(8,drop_remainder=True)
next(iter(tf_testdata))

((<tf.Tensor: shape=(8, 512), dtype=int32, numpy=
  array([[ 101, 2023, 3185, ...,    0,    0,    0],
         [ 101, 3817, 2154, ...,    0,    0,    0],
         [ 101, 2763, 2026, ...,    0,    0,    0],
         ...,
         [ 101, 1045, 2018, ...,    0,    0,    0],
         [ 101, 2023, 1037, ...,    0,    0,    0],
         [ 101, 1037, 6919, ...,    0,    0,    0]])>,
  <tf.Tensor: shape=(8, 512), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]])>),
 <tf.Tensor: shape=(8,), dtype=int32, numpy=array([0, 1, 1, 1, 0, 0, 1, 1])>)

In [68]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.fit(tf_traindata, epochs=3,validation_data=tf_testdata)

Epoch 1/3
8/8 [==============================] - 33s 4s/step - loss: 0.0000e+00 - accuracy: 0.3911 - val_loss: 0.0000e+00 - val_accuracy: 0.5417
Epoch 2/3
8/8 [==============================] - 32s 4s/step - loss: 0.0000e+00 - accuracy: 0.3747 - val_loss: 0.0000e+00 - val_accuracy: 0.4583
Epoch 3/3
8/8 [==============================] - 34s 4s/step - loss: 0.0000e+00 - accuracy: 0.3575 - val_loss: 0.0000e+00 - val_accuracy: 0.5000


## TFBertModel

In [69]:
from transformers import TFBertModel

tfbert_model = TFBertModel.from_pretrained('bert-base-uncased')

input_ids_in = tf.keras.layers.Input(shape=(512,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32') 

embedding_layer = tfbert_model(input_ids_in, attention_mask=input_masks_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(100, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='softmax')(X)
model_bert = tf.keras.Model(inputs=[input_ids_in,input_masks_in], outputs = X)

for layer in model_bert.layers[:3]:
  layer.trainable = False
model_bert.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 768)          0           tf_bert_model[0][0]        

In [70]:
model_bert.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model_bert.fit(tf_traindata, epochs=3,validation_data=tf_testdata)

Epoch 1/3
8/8 [==============================] - 72s 9s/step - loss: 0.9400 - accuracy: 0.4161 - val_loss: 0.5901 - val_accuracy: 0.5417
Epoch 2/3
8/8 [==============================] - 88s 12s/step - loss: 0.4630 - accuracy: 0.4207 - val_loss: 0.5304 - val_accuracy: 0.4583
Epoch 3/3
8/8 [==============================] - 110s 14s/step - loss: 0.6388 - accuracy: 0.4113 - val_loss: 0.4802 - val_accuracy: 0.5000


## TFBertForSequenceClassification

In [71]:
tfbert_cl = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

input_ids_in = tf.keras.layers.Input(shape=(512,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32') 

embedding_layer = tfbert_cl(input_ids_in, attention_mask=input_masks_in)[0]

X = tf.keras.layers.BatchNormalization()(embedding_layer)
X = tf.keras.layers.Dense(100, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='softmax')(X)
model_cl = tf.keras.Model(inputs=[input_ids_in,input_masks_in], outputs = X)

for layer in model_cl.layers[:3]:
  layer.trainable = False
model_cl.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_153']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica ((None, 2),)         109483778   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 2)            8           tf_bert_for_sequence_classi

In [72]:
model_cl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model_cl.fit(tf_traindata, epochs=3,validation_data=tf_testdata)

Epoch 1/3
8/8 [==============================] - 72s 9s/step - loss: 0.0000e+00 - accuracy: 0.4191 - val_loss: 0.0000e+00 - val_accuracy: 0.4583
Epoch 2/3
8/8 [==============================] - 84s 11s/step - loss: 0.0000e+00 - accuracy: 0.3850 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/3
8/8 [==============================] - 121s 15s/step - loss: 0.0000e+00 - accuracy: 0.4269 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
